In [1]:
# !git clone https://github.com/open-mmlab/mmdetection.git

In [2]:
# !git clone https://github.com/SwinTransformer/Swin-Transformer-Object-Detection.git

In [3]:
# from IPython.display import clear_output

# %cd /app/_data/mmdetection
# !pip install -r requirements/build.txt
# !pip install -v -e .  # or "python setup.py develop"
# clear_output()

In [4]:
# from IPython.display import clear_output
# !pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.1/index.html
# !pip install openmim
# !mim install mmdet
# clear_output()

In [5]:
# !pip install mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.1/index.html

In [6]:
# %cd /app/_data/Swin-Transformer-Object-Detection/apex
# # !pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
# !python setup.py install

In [7]:
# %cd /app/_data/Swin-Transformer-Object-Detection
# !pip install -r requirements/build.txt
# !pip install -v -e .  

In [8]:
# %cd /app/_data/Swin-Transformer-Object-Detection
from apex import amp

In [9]:
import mmcv
import mmdet
from mmcv import Config
from mmdet.apis import (
    inference_detector,
    init_detector,
    set_random_seed,
    train_detector,
)
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

In [10]:
# !git clone https://github.com/NVIDIA/apex


In [11]:
# %cd apex
# !pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [12]:
SEED = 37
set_random_seed(SEED, deterministic=False)

In [13]:
%cd /app/_data/Swin-Transformer-Object-Detection
import argparse
import ast
import glob
import json
import os
import re
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
from PIL import Image
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
import torch
import torchvision

/app/_data/Swin-Transformer-Object-Detection


In [20]:
def df_to_coco(df):
    images = []
    annotations = []
    img_id = 0
    annotation_id = 0
    categories = [
        {
            "id": 0,
            "name": "starfish",
        },
    ]
    for ix in tqdm(df.index.tolist()):
        img = Image.open(df.loc[ix, "img_path"])
        w, h = img.size
        bboxes = df.loc[ix, "annotations"]
        img_name = f'{df.loc[ix, "image_id"]}.jpg'
        img_dict = {
            "id": img_id,
            "file_name": img_name,
            "height": h,
            "width": w,
        }
        images.append(img_dict)
        if len(bboxes) != 0:
            for i in range(len(bboxes)):
                bbox = bboxes[i]
                bbox = [bbox["x"], bbox["y"], bbox["width"], bbox["height"]]
                annotations_dict = {
                    "id": annotation_id,
                    "image_id": img_id,
                    "category_id": 0,
                    "bbox": bbox,
                    "area": bbox[2] * bbox[3],
                    "segmentation": [],
                    "iscrowd": 0,
                }
                annotations.append(annotations_dict)
                annotation_id += 1
        img_id += 1
    coco_dict = {"categories": categories, "images": images, "annotations": annotations}
    return coco_dict

In [15]:
TRAIN_DF_PART = "/app/_data/tensorflow-great-barrier-reef/train.csv"
IMAGE_FOLDER = "images"
LABEL_FOLDER = "labels"

In [16]:
with open("/app/_data/sequences.json", "r") as f:
    seq_dict = json.load(f)

In [17]:
df = pd.read_csv(TRAIN_DF_PART)
df["img_path"] = (
    "/app/_data/tensorflow-great-barrier-reef/train_images/video_"
    + df.video_id.astype("str")
    + "/"
    + df.video_frame.astype("str")
    + ".jpg"
)
df["annotations"] = df["annotations"].apply(lambda x: ast.literal_eval(x))
df["len_annotation"] = df["annotations"].str.len()
df["image_id"] = df["image_id"].str.replace("-", "_", regex=True)
df["new_img_path"] = f"/app/_data/{IMAGE_FOLDER}/" + df["image_id"] + ".jpg"
df["label"] = df["len_annotation"].apply(lambda x: 0 if x == 0 else 1)
df["no_label"] = df["len_annotation"].apply(lambda x: True if x == 0 else False)
R = df[df["len_annotation"] == 0].shape[0] / df[df["len_annotation"] != 0].shape[0]
df["label_change"] = df["label"] & df["no_label"].shift(1) & df["no_label"].shift(
    2
) | df["no_label"] & df["label"].shift(1) & df["label"].shift(2)
df["sequense_change"] = df["sequence"] != df["sequence"].shift(1)
df["start_subseq"] = df["sequense_change"] | df["label_change"]
df.loc[df.index[-1], "start_subseq"] = True
df["start_subseq"].sum()
start_idx = 0
for subsequence_id, end_idx in enumerate(df[df["start_subseq"]].index):
    df.loc[start_idx:end_idx, "subsequence_id"] = subsequence_id
    start_idx = end_idx

df["subsequence_id"] = df["subsequence_id"].astype(int)
df["subsequence_id"].nunique()

138

137

# train_test_split

## video_id

In [18]:
VIDEO_ID = 2
train  = pd.concat(
    [
        df.query("video_id!=@VIDEO_ID and len_annotation!=0"),
        df.query("video_id!=@VIDEO_ID and len_annotation==0").sample(
            int(
                df.query("video_id!=@VIDEO_ID and len_annotation!=0").shape[0]
                * 0.07
            )
        ),
    ]
).sample(frac = 1)
val = df.query("video_id==@VIDEO_ID").sample(frac = 1)

In [19]:
%%writefile /app/_data/for_mmdet/labels.txt 
starfish

Overwriting /app/_data/for_mmdet/labels.txt


In [21]:
train_dict = df_to_coco(train)
with open('/app/_data/for_mmdet/coco_train.json', 'w') as f:
    json.dump(train_dict, f, ensure_ascii=True, indent=4)
val_dict = df_to_coco(val)
with open('/app/_data/for_mmdet/coco_val.json', 'w') as f:
    json.dump(val_dict, f, ensure_ascii=True, indent=4)

100% 4538/4538 [00:00<00:00, 6284.80it/s]
100% 8561/8561 [00:01<00:00, 6599.80it/s]


# Model Config

In [22]:
cfg = Config.fromfile(
    "/app/_data/Swin-Transformer-Object-Detection/configs/swin/cascade_mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py"
)

In [23]:
# cfg

In [24]:
model_name = "cascade_mask_rcnn_swin_tyny_patch_1"
# Folder to store model logs and weight files
work_dir = f"/app/_data/for_mmdet/{model_name}"
cfg.work_dir = work_dir
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

In [25]:
for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1

In [26]:
cfg.model.roi_head.mask_head.num_classes = 1

In [27]:
cfg.model.backbone

{'type': 'SwinTransformer',
 'embed_dim': 96,
 'depths': [2, 2, 6, 2],
 'num_heads': [3, 6, 12, 24],
 'window_size': 7,
 'mlp_ratio': 4.0,
 'qkv_bias': True,
 'qk_scale': None,
 'drop_rate': 0.0,
 'attn_drop_rate': 0.0,
 'drop_path_rate': 0.2,
 'ape': False,
 'patch_norm': True,
 'out_indices': (0, 1, 2, 3),
 'use_checkpoint': False}

In [28]:
cfg.optimizer

{'type': 'AdamW',
 'lr': 0.0001,
 'betas': (0.9, 0.999),
 'weight_decay': 0.05,
 'paramwise_cfg': {'custom_keys': {'absolute_pos_embed': {'decay_mult': 0.0},
   'relative_position_bias_table': {'decay_mult': 0.0},
   'norm': {'decay_mult': 0.0}}}}

In [29]:
cfg.runner.max_epochs = 30

In [30]:
cfg.lr_config

{'policy': 'step',
 'warmup': 'linear',
 'warmup_iters': 500,
 'warmup_ratio': 0.001,
 'step': [27, 33]}

In [31]:
# cfg.lr_config = dict(
#     policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
#     by_epoch=False,
#     warmup='linear', # The warmup policy, also support `exp` and `constant`.
#     warmup_iters=500, # The number of iterations for warmup
#     warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
#     min_lr=1e-07)

In [32]:
cfg.log_config.interval = 200

In [33]:
cfg.data.samples_per_gpu=8
cfg.data.workers_per_gpu=4

In [34]:
cfg.dataset_type

'CocoDataset'

In [35]:
cfg.gpu_ids = [0]
cfg.classes = '/app/_data/for_mmdet/labels.txt'

In [36]:
cfg.data.train.ann_file =  "/app/_data/for_mmdet/coco_train.json"
cfg.data.train.img_prefix =  "/app/_data/images/"
cfg.data.train.classes = cfg.classes

In [37]:
cfg.data.val.ann_file =  "/app/_data/for_mmdet/coco_val.json"
cfg.data.val.img_prefix =  "/app/_data/images/"
cfg.data.val.classes = cfg.classes

In [38]:
cfg.data.test = cfg.data.val

In [39]:
cfg.seed = SEED

In [40]:
cfg.evaluation.metric # = ['bbox']

['bbox', 'segm']

In [41]:
import wandb
wandb.login()

wandb: Currently logged in as: tatanko (use `wandb login --relogin` to force relogin)


True

In [42]:
cfg.log_config.hooks

[{'type': 'TextLoggerHook'}]

In [43]:
cfg.log_config.hooks = [dict(type='TextLoggerHook'),
                        dict(type='WandbLoggerHook',
                             init_kwargs=dict(project='cascade_101',
                                              name=f'exp-{model_name}',
                                              entity='tatanko'))]

In [44]:
cfg.data.train.pipeline[1]['with_mask'] = False

In [45]:
cfg.data.test.pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'MultiScaleFlipAug',
  'img_scale': (1333, 800),
  'flip': False,
  'transforms': [{'type': 'Resize', 'keep_ratio': True},
   {'type': 'RandomFlip'},
   {'type': 'Normalize',
    'mean': [123.675, 116.28, 103.53],
    'std': [58.395, 57.12, 57.375],
    'to_rgb': True},
   {'type': 'Pad', 'size_divisor': 32},
   {'type': 'ImageToTensor', 'keys': ['img']},
   {'type': 'Collect', 'keys': ['img']}]}]

In [46]:
cfg.data.train.pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': False},
 {'type': 'RandomFlip', 'flip_ratio': 0.5},
 {'type': 'AutoAugment',
  'policies': [[{'type': 'Resize',
     'img_scale': [(480, 1333),
      (512, 1333),
      (544, 1333),
      (576, 1333),
      (608, 1333),
      (640, 1333),
      (672, 1333),
      (704, 1333),
      (736, 1333),
      (768, 1333),
      (800, 1333)],
     'multiscale_mode': 'value',
     'keep_ratio': True}],
   [{'type': 'Resize',
     'img_scale': [(400, 1333), (500, 1333), (600, 1333)],
     'multiscale_mode': 'value',
     'keep_ratio': True},
    {'type': 'RandomCrop',
     'crop_type': 'absolute_range',
     'crop_size': (384, 600),
     'allow_negative_crop': True},
    {'type': 'Resize',
     'img_scale': [(480, 1333),
      (512, 1333),
      (544, 1333),
      (576, 1333),
      (608, 1333),
      (640, 1333),
      (672, 1333),
      (704, 1333),
      (736, 1333),
      (768, 1333),
      (800, 1333)

In [47]:
cfg.data.train.pipeline = cfg.train_pipeline

In [48]:
model = build_detector(
    cfg.model, train_cfg=cfg.get("train_cfg"), test_cfg=cfg.get("test_cfg")
)
model.init_weights()

In [49]:
train_dataset = build_dataset(cfg.data.train)

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


In [50]:
# train_dataset.img_ids

In [51]:
train_detector(model, train_dataset, cfg, distributed=False, validate=True)

2022-02-15 11:03:26,247 - mmdet - INFO - Start running, host: root@53ab8357cb6e, work_dir: /app/_data/for_mmdet/cascade_mask_rcnn_swin_tyny_patch_1
2022-02-15 11:03:26,249 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(ABOVE_NORMAL) DistOptimizerHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/app/_data/Swin-Transformer-Object-Detection/mmdet/datasets/custom.py", line 193, in __getitem__
    data = self.prepare_train_img(idx)
  File "/app/_data/Swin-Transformer-Object-Detection/mmdet/datasets/custom.py", line 216, in prepare_train_img
    return self.pipeline(results)
  File "/app/_data/Swin-Transformer-Object-Detection/mmdet/datasets/pipelines/compose.py", line 40, in __call__
    data = t(data

In [55]:
for i in train_dataset:
    i